INFO 2950 - FINAL PROJECT (Avni, Aryana, and Ishneet)

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [85]:
aqi = pd.read_csv("daily_aqi_by_county_2018.csv")
pop = pd.read_csv("PopulationEstimates.csv")
inc = pd.read_csv("lapi1119.csv")

In [86]:
aqi_pivot = pd.pivot_table(aqi, values = 'AQI', columns = ['Date'], index = ['State Name','county Name'], aggfunc=np.sum)
aqi_pivot = aqi_pivot.reset_index(level=['State Name','county Name'])

aqi_pivot['avg_aqi'] = round(aqi_pivot.drop(['State Name','county Name'], axis=1).apply(lambda x: x.mean(), axis=1),2)

aqi_clean = aqi_pivot[['State Name', 'county Name', 'avg_aqi']]
aqi_clean.head()

Date,State Name,county Name,avg_aqi
0,Alabama,Baldwin,36.83
1,Alabama,Clay,28.99
2,Alabama,Colbert,35.64
3,Alabama,DeKalb,36.72
4,Alabama,Elmore,35.33


This dataset doesn't require much cleaning. When we produce our own dataset, we will have to average out daily AQIs into a yearly AQI, in order to be consistent with the rest of our dataset. Since the per capita income is calculated every July 1st, averaging AQI for every July 1st (or approximately that date) will help keep the data consistent across the board. We cleaned this dataset by createing a pivot table based off of the dates and then found an average over the whole year for each county. This average column is the only important one for us. 

There are a few things to clean about the above dataset. The NaN values result because the rows are left blank in the raw dataset because row 0 signifies a change in state or a description in the beginning of the dataset. This needs to be cleaned. Additionally, the column headings are not named like they are in the raw dataset. Instead, the column names are actually row 1's value. Thus, rows 0 and the current column headings need to be dropped. Furthermore, we only need population estimates for 2018, so our dataset will include only that column. 

In [87]:
#dropping the first row of the population dataset
pop_clean = pop.iloc[1:]
#setting the column headings to row 1
pop_clean.columns = pop_clean.iloc[0]
pop_clean = pop_clean.drop(1, axis = 0)
#now we have proper column headings. The only relevant information we want from this dataset is the population estimate

In [88]:
pop_2018 = pop_clean[['State', 'Area_Name','POP_ESTIMATE_2018']]
#print("The number of NaN values in pop_2018 is", pop_2018.isnull().sum().sum()) 
#pop_clean.head(3274) #this now tell us that our dataset had an extra row of values that are not needed
#dropping the extra row of values
pop_2018 = pop_2018[:-1]
pop_2018.head()

1,State,Area_Name,POP_ESTIMATE_2018
2,US,United States,"327,167,434"
3,AL,Alabama,"4,887,871"
4,AL,Autauga County,"55,601"
5,AL,Baldwin County,"218,022"
6,AL,Barbour County,"24,881"


pop_2018 is now the cleaned, usable series for our own dataset. It includes population estimates for 2018 only and NaN values have been excluded (which was only the last row that was added accidentally from the source so this doesn't affect our sample size). What is important to note in this series is that it includes totals for each state. 

We see the same problem with the income dataset as we did with the population estimates dataset. The column names should be row 0 values and NaN values are purposely left blank in the raw dataset to improve readability.

In [96]:
inc_clean = inc
#setting the column headings to row 1
inc_clean.columns = inc_clean.iloc[0]
inc_clean.columns = ['State','County', 'Per capita personal income 2016', 'Per capita personal income 2017', 
                     'Per capita personal income 2018', 'Rank in State 2018', 'Percent change 2017', 'Percent change 2018',
                    'Percent change in rank 2018']
inc_clean = inc_clean.iloc[3:]
#There is again an issue with NaN values, this time there are NaN values after each state because the original data set left the row blank

In [98]:
#dropping all NaN rows from this dataset
inc_clean = inc_clean.dropna()

#resetting the index to a sequential order
inc_clean = inc_clean.reset_index(drop=True)

#extracting per capital personal income in 2018
inc_2018 = inc_clean[['State','County', 'Per capita personal income 2018']]
inc_2018.head(75)

,State,County,Per capita personal income 2018
0,Alabama,Alabama,"42,238"
1,Alabama,Autauga,"41,618"
2,Alabama,Baldwin,"45,596"
3,Alabama,Barbour,"35,199"
4,Alabama,Bibb,"30,254"
...,...,...,...
70,Alaska,Aleutians West Census Area,"54,385"
71,Alaska,Anchorage Municipality,"66,510"
72,Alaska,Bethel Census Area,"43,500"
73,Alaska,Bristol Bay Borough,"143,575"


inc_clean is now a cleaned dataset, and inc_2018 contains personal per capital income data from 2018 we need to put in our dataset